In [1]:
import openai
import pandas as pd
import ast  
import re

In [2]:
def get_api_key(file_path):
    with open(file_path, "r") as file:
        return file.read().strip()

In [3]:
def extract_valid_list(response_text):
    
    response_text = re.sub(r"```[a-zA-Z]*\n", "", response_text)  
    response_text = response_text.strip("`") 

    match = re.search(r"\[.*?\]", response_text, re.DOTALL)

    if match:
        list_text = match.group(0)  
        try:
            extracted_list = ast.literal_eval(list_text) 
            if isinstance(extracted_list, list):
                return extracted_list
        except Exception:
            pass  
    
    return []  

In [4]:
def generate_ner_tags(sentence, client):
    
    user_prompt = f"""
    You are a business process modelling expert, tasked with identifying which mentions of certain process relevant elements in textual descriptions of business processes belong to the same entity. 

    Allowed NER tags and their explanations:
    - O: An O tag indicates that a token belongs to no chunk.
    - B-Actor: This tag indicates the beginning of an Actor chunk.
    - I-Actor: This tag indicates that the token is inside an Actor chunk.
    - B-Activity: This tag indicates the beginning of an Activity chunk.
    - I-Activity: This tag indicates that the token is inside an Activity chunk.
    - B-Activity Data: This tag indicates the beginning of an Activity Data chunk.
    - I-Activity Data: This tag indicates that the token is inside an Activity Data chunk.
    - B-Further Specification: This tag indicates the beginning of a Further Specification chunk.
    - I-Further Specification: This tag indicates that the token is inside a Further Specification chunk.
    - B-XOR Gateway: This tag indicates the beginning of a XOR Gateway chunk.
    - I-XOR Gateway: This tag indicates that the token is inside a XOR Gateway chunk.
    - B-Condition Specification: This tag indicates the beginning of a Condition Specification chunk.
    - I-Condition Specification: This tag indicates that the token is inside a Condition Specification chunk.
    - B-AND Gateway: This tag indicates the beginning of an AND Gateway chunk.
    - I-AND Gateway: This tag indicates that the token is inside an AND Gateway chunk.

    An example of this could look like this:
    Sentence:
    ['The', 'post', 'is', 'advertised', ',', 'applicants', 'apply', ',', 'the', 'applications', 'are', 'checked', 'and', 'the', 'post', 'is', 'filled', '.']
    NER_tags:
    ['B-Activity Data', 'I-Activity Data', 'O', 'B-Activity', 'O', 'B-Actor', 'B-Activity', 'O', 'B-Activity Data', 'I-Activity Data', 'O', 'B-Activity', 'O', 'B-Activity Data', 'I-Activity Data', 'O', 'B-Activity', 'O']
    
    
    Now, label each token in the following sentence with one of the above NER tags:
    
    Sentence: {sentence}
    
    Output ONLY a Python-style list of NER tags. Do not include any explanations or additional text.
    """

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "user", "content": user_prompt}
        ],
        temperature=0.0,
        max_tokens=1024
    )

    response_text = response.choices[0].message.content.strip()
    return extract_valid_list(response_text)

In [5]:
def process_dataset(input_file, output_file, api_key_file):
    api_key = get_api_key(api_key_file)
    client = openai.OpenAI(api_key=api_key)  
    
    dataset = pd.read_csv(input_file)

    def process_row(row):
        sentence = " ".join(ast.literal_eval(row["tokens"]))
        try:
            return generate_ner_tags(sentence, client)
        except Exception:
            return []  

    dataset["gpt4o-mini_ner_tags"] = dataset.apply(process_row, axis=1)
    
    dataset.to_csv(output_file, index=False)

In [6]:
if __name__ == "__main__":
    input_file = "PET-Customized.csv"
    output_file = "PET-Customized-GPT4o-mini.csv"
    api_key_file = "./API_Key/OpenAI.txt"

    process_dataset(input_file, output_file, api_key_file)